In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

# Prepare the datasets

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Helper functions

In [4]:
sess = tf.InteractiveSession()

In [5]:
# init weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [6]:
# init bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
# conv2d
def conv2d(x,W): 
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# x ---->  [batch, in_height, in_width, in_channels] ---->data_format=[NHWC]
# W ---->  filter / kernel ----> [filter_height, filter_width, in_channels, out_channels], out_channels --> how many filters
# strides = [1, stride, stride, 1]
# Returns:
#   A `Tensor`. Has the same type as `input`.
#   A 4-D tensor. The dimension order is determined by the value of
#   `data_format`


In [8]:
# max pooling
def max_pooling_2by2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# x ---> [batch, height, width, channels]` and type `tf.float32`---->data_format='NHWC'
# ksize ----> h,w of input
# Returns:  A `Tensor` with type `tf.float32`.  The max pooled output tensor.

In [9]:
# convolutional layer
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]]) 
    return tf.nn.relu(conv2d(input_x, W) + b)
#     shape = [batch, height, width, channels], shape[3] = channels. How many channels how many bias


In [10]:
# fully connected layer
def fullyconnected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b
    
#     input_layer is 4D tensor, NHWC, shape[1] is Height

# Build a model

In [11]:
# placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32,shape=[None, 10])

In [12]:
# layers 
# conv_1
# conv_1_maxpooling
# conv_2
# conv_2_maxpooling
# fully_1
# fully_1_dropout
# y_pred

x_image = tf.reshape(x, [-1,28,28,1])

In [13]:
# The first convolutional layer
conv_1 = convolutional_layer(x_image, shape=[5,5,1,32])
conv_1_maxpooling = max_pooling_2by2(conv_1)

In [14]:
# The second convolutional layer
conv_2 = convolutional_layer(conv_1_maxpooling,shape=[5,5,32,32])
conv_2_maxpooling = max_pooling_2by2(conv_2)


In [15]:
# flatten
conv_2_flat = tf.reshape(conv_2_maxpooling,[-1, 7*7*32])

# The first fully connected layer
fully_1 = tf.nn.relu(fullyconnected_layer(conv_2_flat,512))

In [16]:
# dropout layer
hold_prob = tf.placeholder(tf.float32)
fully_1_dropout = tf.nn.dropout(fully_1,keep_prob=hold_prob)

In [17]:
# The second fully connected layer
y_pred = fullyconnected_layer(fully_1_dropout,10)

In [18]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [19]:
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

# Train on the datasets

In [26]:
init = tf.global_variables_initializer()

In [27]:
saver = tf.train.Saver()

In [28]:
steps = 1000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')
        
    saver.save(sess,'models/mnist_cnn.ckpt')

Currently on step 0
Accuracy is:
0.1397


Currently on step 100
Accuracy is:
0.7907


Currently on step 200
Accuracy is:
0.8884


Currently on step 300
Accuracy is:
0.9115


Currently on step 400
Accuracy is:
0.9261


Currently on step 500
Accuracy is:
0.9333


Currently on step 600
Accuracy is:
0.9436


Currently on step 700
Accuracy is:
0.9458


Currently on step 800
Accuracy is:
0.9496


Currently on step 900
Accuracy is:
0.9532


